     |████████████████████████████████| 4.3 MB 5.1 MB/s 
  Attempting uninstall: Pillow
    Found existing installation: Pillow 7.1.2
    Uninstalling Pillow-7.1.2:
      Successfully uninstalled Pillow-7.1.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
albumentations 0.1.12 requires imgaug<0.2.7,>=0.2.5, but you have imgaug 0.2.9 which is incompatible.


In [88]:
from craft_text_detector import (
    read_image,
    load_craftnet_model,
    load_refinenet_model,
    get_prediction,
    export_detected_regions,
    export_extra_results,
    empty_cuda_cache
)
import easyocr
import pytesseract
try:
    from PIL import Image
except ImportError:
    import Image
import numpy as np
import re
import io
import re
from scipy.ndimage import interpolation as inter
import cv2

reader = easyocr.Reader(['en'], gpu=True)
refine_net = load_refinenet_model(cuda=True)
craft_net = load_craftnet_model(cuda=True)
cstm_config = r"-l eng+frn --psm 10 --oem 3 tessedit_char_whitelist=ABCDEFGHIJKLMNOPQRSTUVWXYZ0123456789"

def correct_skew(image, delta=0.3, limit=10):
    def determine_score(arr, angle):
        data = inter.rotate(arr, angle, reshape=False, order=0)
        histogram = np.sum(data, axis=1)
        score = np.sum((histogram[1:] - histogram[:-1]) ** 2)
        return histogram, score

    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    thresh = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)[1] 

    scores = []
    angles = np.arange(-limit, limit + delta, delta)
    for angle in angles:
        _, score = determine_score(thresh, angle)
        scores.append(score)

    best_angle = angles[scores.index(max(scores))]

    (h, w) = image.shape[:2]
    center = (w // 2, h // 2)
    M = cv2.getRotationMatrix2D(center, best_angle, 1.0)
    rotated = cv2.warpAffine(image, M, (w, h), flags=cv2.INTER_CUBIC, \
              borderMode=cv2.BORDER_REPLICATE)

    return rotated


In [95]:
image = np.array(Image.open('/content/СТС_16.jpeg'))

h,w = image.shape[:2]
print(h,w)
img = cv2.resize(image, (3100, 3500), fx=0.5, fy=0.3)
image = read_image(img)


prediction_result = get_prediction(
    image=image,
    craft_net=craft_net,
    refine_net=refine_net,
    text_threshold=0.7,
    link_threshold=0.4,
    low_text=0.4,
    cuda=True,
    long_size=560
)

idx = 0
ocr_text = []
ocr_text1 = []
boxes = prediction_result["boxes"]
for box in boxes[4:]:
    idx+=1
    x,y,w,h = cv2.boundingRect(box)
    resize_image = image[y:y+h, x:x+w]
    new_image1 = correct_skew(resize_image)
    gray = cv2.cvtColor(new_image1, cv2.COLOR_BGR2GRAY)
    result_easy = reader.readtext(gray, detail=0)
    result_tes = pytesseract.image_to_string(gray,config=cstm_config)
    ocr_text.append(result_easy)
    ocr_text1.append(result_tes)
  

def get_birthday(data):
    date_birth = re.findall(r"[A-Z0-9]{15,17}", str(data))
    return date_birth

def get_number_plate(data):
    number = re.findall(r"[A-Z0-9]{7,9}", str(data))
    return number


tes_easy = ocr_text+ocr_text1

series_number = list(map(get_birthday, tes_easy))
texting = list(filter(None, series_number))

print(texting)

1098 792


/usr/local/lib/python3.7/dist-packages/craft_text_detector/craft_utils.py:415: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  polys = np.array(polys)
/usr/local/lib/python3.7/dist-packages/craft_text_detector/predict.py:110: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  polys_as_ratio = np.array(polys_as_ratio)


[['JTHBJ1GG402016817'], ['JTHBJ1G6402016817'], ['ITHBI166402016817'], ['JTHBJ1GG402016817']]


In [97]:
print(len(ocr_text))

17


In [33]:
import json
import requests
dict_keys = {"01":"Грузовой автомобиль бортовой",
        "02":"Грузовой автомобиль шасси",
        "03":"Грузовой автомобиль фургоны",
        "04":"Грузовой автомобиль тягачи седельной",
        "05":"Грузовой автомобиль самосвалы",
        "06":"Грузовой автомобиль рефрижераторы",
        "07":"Грузовой автомобиль цистерны",
        "08":"Грузовой автомобиль с гидроманипулятором",
        "09":"Грузовой автомобиль прочие",
        "21":"Легковой автомобиль универсал",
        "22":"Легковой автомобиль комби (хэтчбек)",
        "23":"Легковой автомобиль седан",
        "24":"Легковой автомобиль лимузин",
        "25":"Легковой автомобиль купе",
        "26":"Легковой автомобиль кабриолет",
        "27":"Легковой автомобиль фаэтон",
        "28":"Легковой автомобиль пикап",
        "29":"Легковой автомобиль прочие",
        "41":"Автобус длиной не более 5 м",
        "42":"Автобус длиной более 5 м, но не более 8 м",
        "43":"Автобус длиной более 8 м, но не более 12 м",
        "44":"Автобус сочлененные длиной более 12 м",
        "49":"Автобус прочие",
        "51":"Специализированные автомобили автоцистерны",
        "52":"Специализированные автомобили санитарные",
        "53":"Специализированные автомобили автокраны",
        "54":"Специализированные автомобили заправщики",
        "55":"Специализированные автомобили мастерские",
        "56":"Специализированные автомобили автопогрузчики",
        "57":"Специализированные автомобили эвакуаторы",
        "58":"Специализированные пассажирские транспортные средства",
        "59":"Специализированные автомобили прочие",
        "71":"Мотоциклы",
        "72":"Мотороллеры и мотоколяски",
        "73":"Мотовелосипеды и мопеды",
        "74":"Мотонарты",
        "80":"Прицепы самосвалы",
        "81":"Прицепы к легковым автомобилям",
        "82":"Прицепы общего назначения к грузовым автомобилям",
        "83":"Прицепы цистерны",
        "84":"Прицепы тракторные",
        "85":"Прицепы вагоны-дома передвижные",
        "86":"Прицепы со специализированными кузовами",
        "87":"Прицепы трейлеры",
        "88":"Прицепы автобуса",
        "89":"Прицепы прочие",
        "91":"Полуприцепы с бортовой платформой",
        "92":"Полуприцепы самосвалы",
        "93":"Полуприцепы фургоны",
        "95":"Полуприцепы цистерны",
        "99":"Полуприцепы прочие",
        "31":"Трактора",
        "32":"Самоходные машины и механизмы",
        "33":"Трамваи",
        "34":"Троллейбусы",
        "35":"Велосипеды",
        "36":"Гужевой транспорт",
        "38":"Подвижной состав железных дорог",
        "39":"Иной"}
params = {'vin': 'XWEPC81ADE0009527',
    'checkType':'history'}

resp = requests.post("https://сервис.гибдд.рф/proxy/check/auto/history", data=params)

data = json.loads(resp.text)

items = data['RequestResult']['vehicle']


model = items['model']
vin = items['vin']
year = items['year']
category = items['category']
type = items['type']
type_ts = dict_keys.get(type)
engine_Hp = items['powerHp']
engine_powerKwt = items['powerKwt']
engine = f'({engine_powerKwt}){engine_Hp}'
data = {
    'number': 'number',
    'vin': vin,
    'model':model,
    'type':type_ts,
    'categoty':category,
    'year':year,
    'engine':engine,
    'date':''
}
print(data)

KeyboardInterrupt: ignored